# Load the Packages

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Setting up Shapefiles

[Shapefiles](https://en.wikipedia.org/wiki/Shapefile) are a collection of file formats that are commonly used for working with geospatial data.

Shapefiles can take the form of various *geometries*: points, lines, polylines, and polygons. We can use these geometries as containers onto which we can attach attributes and features the same way we would add columns to a table.

### Coordinate Reference System (CRS)

When working with geospatial data, we need to make sure that all of our data uses the same [coordinate reference system (crs)](https://en.wikipedia.org/wiki/Spatial_reference_system). CRS varies from region to region, but the most commonly used one is [WGS-84](https://epsg.io/4326) (latitude/longitude).

### Census Tracts

Census tracts are the second smallest geographical unit used by the Census Bureau. Each tract has a 6-digit unique identifier that can be extended to include information specifying the county and state. In NYC, boroughs are coterminous with counties, meaning they have identical geographic boundaries. When working with NYC census data, census tracts are uniquely identified by a 6-digit code: the first digit represents the borough/county, followed by six digits for the tract.

**NOTE:** It's important that you use the 7-digit code (usually called `boro_ct2010` or something similar) and not the 6-digit `ct2010` code. Census tracts in different counties can have overlapping IDs!

In [ ]:
#load the shapefile located in 'data/nyc_tracts/ct2010.shp'


#set the crs to 'epsg:4326'


#rename the census tract id column 'boro_ct201' to 'tract' for easier merging


#convert to a string for merging purposes



# Merging Population Data onto Shapefiles

The [United States Census Bureau](https://www.census.gov/data.html) makes all of its data public. You can either download csvs from the [American Fact Finder](https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml) or use the [Census API](https://www.census.gov/data/developers/data-sets.html).

NYC Planning also has a nice [NYC Population FactFinder](https://popfactfinder.planning.nyc.gov/#12.25/40.724/-73.9868) tool which makes it easier to interact with NYC census data.

There are also several [python packages](https://github.com/datamade/census) that make working with census data much easier.

*Pulling census data is not the easiest process, so I'm giving you a nice clean dataset of population data taken from the 2010 census.*

**File location:** "data/pop2010.csv"

In [ ]:
pop = pd.read_csv('data/pop2010.csv', dtype={'tract': str, 'pop': int})
pop.head()

**IMPORTANT:** We're going to perform a *left join* by merging the population data onto the census tract shapefiles. This is necessary when working with geopandas because if done the other way around, the GeoDataFrame format will break and we won't be able to map our data.

[Here's an article](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html) that does a pretty good job of explaining joins in python if you want to learn more.

In [ ]:
#merge the 'pop' dataset onto tracts


# Plotting

Now that we have our population data attached to our shapefiles, we can already go ahead and do some plotting. Geopandas plotting features are built on top of matplotlib, so we can use it's syntax. Whether or not that is a good thing is up to you to decide.

Matplotlib has a decent selection of [colormaps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html) that you can choose from.

It's possible to [create your own custom colormaps](https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html), but that takes time and goes a bit beyond the scope of this workshop.

In [ ]:
# #set-up the canvas
fig, ax = plt.subplots(figsize = (16,16))

# #plot a choropleth map using data from the 'pop' column of the 'tracts' dataset
# #you can specify what colormap you want to use with the 'cmap' parameter
# #sometimes you might want to reverse the direction of colormap by adding '_r' to the end of the colormap's name


# #we can also turn off the axes to make things look cleaner


# #this last function displays your map
plt.show()

# Noise Complaints

Now that we have our census tracts and population data set up, let's move on to our 311 data. The [311 Service Requests](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) dataset is one of the most popular datasets on the NYC Open Data Portal.

For those unfamiliar with [311](https://www1.nyc.gov/311/our-mission.page), it's a service offered by the city government which allows New Yorkers to report non-emergencies like potholes or noise complaints.

This dataset has been filtered for commercial noise complaints placed between the hours of 7:00pm to 5:00am.

In [ ]:
noise = pd.read_csv('data/noise311.csv')
noise.head()

Once we've loaded our noise complaint data, we need to convert it into a geodataframe so that we can perform a *spatial join*. In order to do so, we have to zip the `lon` and `lat` columns together and convert it to a `Point` object.

In [ ]:
#zip the lon/lat columns


#convert the coordinate pair to a Point object


#convert the DataFrame into a GeoDataFrame


#don't forget to set the crs!


# Layering Plots

We can layer different GeoDataFrame plots on top of one another. Let's try layering the 311 points on top of our census tracts.

In [ ]:
#create the canvas
fig, ax = plt.subplots(figsize = (20,16), facecolor = "#080808")

#plot the basemap


#plot the noise complaints


#turn off the axis
ax.axis('off')

#optional: save the image
# plt.savefig('noise_complaints.png', dpi = 300, facecolor = "#080808")

#display the plot
plt.show()

# Binning Data Using Spatial Joins

Visualizing coordinate points is pretty and insightful, but in order to draw clearer comparisons across different areas, we need to group these points by using `gpd.sjoin()` to perform a *spatial join* that groups noise complaints by census tract.

**NOTE:** This is **very** computationally expensive and can take an unreasonably long time as the size of your data increases. If you're feeling really ambitious and want to learn more about performing spatial joins with large datasets, you should look into [R-tree indexing](https://geoffboeing.com/2016/10/r-tree-spatial-index-python/).

In [ ]:
#create a new dataset by performing a spatial join using the noise and tracts GeoDataFrames


# Counting Complaints per Tract

Now that we've attached a census tract to each complaint, we can use the `value_counts()` function to count the number of complaints grouped by the `tract` column.

In [ ]:
#count by tract

#convert to a DataFrame and fix column names


# Merge Grouped Complaints onto Census Tracts

We now have the number of complaints for each census tract, which means all we need to do is perform another left join onto the original `tracts` GeoDataFrame.

In [ ]:
#create a new GeoDataFrame that merges the counts onto tracts


#any NaN values are zeros in this situation


#remove parks & census tracts without people -- these will skew our data


#create a new column for per-capita complaints



# Plotting Late-Night Commercial Noise Complaints by Tract

Now lets plot our map of noise complaints on top of the basemap.

In [ ]:
fig, ax = plt.subplots(figsize = (20,16))

#your code here


ax.axis('off')
plt.show()

Doesn't look particularly great, does it? This is because most census tracts have a similar level of per-capita noise complaints. Let's try looking at outliers instead...

# Plotting Outliers

This is about as statistics-heavy as this workshop is going to get.

We're interested in identifying areas that experience an unusually-high level of per-capita noise complaints--in other words: outliers. There are many different ways to calculate outliers, but the most common is to find values that are 2 or 3 **standard deviations** away from the **mean** (depending on whether you are using a 95% or 99% confidence interval).

**NOTE:** *If nothing written above makes any sense to you -- don't worry. This is basically a way of looking at how extreme certain data points are compared to the rest of the data.*

In [ ]:
#create a variable for the mean, standard deviation, and an upper-bound to identify outliers

#create a new dataset that filters only those that are greater than or equal to our upper-bound


Now that we have our outliers, we can plot them on top of our basemap using a single color.

In [ ]:
#set up the canvas
fig, ax = plt.subplots(figsize = (20,16))

#plot the basemap


#plot the outliers



ax.axis('off')

#save the figure
plt.savefig('noise_complaint_outliers.png', dpi = 300, transparent = True)

plt.show()

# Other Tools

Geopandas is a good tool for learning how to work with data and prepare it for visualization, but there are a number of other tools that might be better suited for your needs. If you're looking to get into interactive, web-based visualizations, then [CARTO](https://carto.com/), [Leaflet](https://leafletjs.com/), and [Mapbox](https://www.mapbox.com/) are good places to start.

If you're feeling super scientific and love Windows XP style UIs, then [ArcGIS](http://www.arcgis.com/index.html) is the gold-standard GIS software.